In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import utils

In [3]:
#adding cal-itp id 
bbb = 300
st = 290

In [4]:
pd.set_option("display.max_columns", 100)


In [5]:
rt_sched = utils.read_data()

In [6]:
rt_sched.head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
0,290,SamTrans,1,14,14,2022-05-04,2,0,0.0,Wednesday,May
1,290,SamTrans,1,14,14,2022-05-04,2,0,0.0,Wednesday,May
2,290,SamTrans,1,14,14,2022-05-04,2,0,0.0,Wednesday,May
3,290,SamTrans,1,17,17,2022-06-23,33,0,0.0,Thursday,June
4,290,SamTrans,1,17,17,2022-06-19,16,0,0.0,Sunday,June


In [7]:
rt_sched.tail()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
11216,300,Big Blue Bus,0,3479,1,2022-06-17,194,194,1.0,Friday,June
11217,300,Big Blue Bus,0,3479,1,2022-05-12,195,195,1.0,Thursday,May
11218,300,Big Blue Bus,0,3479,1,2022-05-03,196,196,1.0,Tuesday,May
11219,300,Big Blue Bus,0,3479,1,2022-05-17,196,196,1.0,Tuesday,May
11220,300,Big Blue Bus,0,3479,1,2022-06-21,197,197,1.0,Tuesday,June


In [8]:
rt_sched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11221 entries, 0 to 11220
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   calitp_itp_id      11221 non-null  int64         
 1   agency_name        7938 non-null   object        
 2   calitp_url_number  11221 non-null  int64         
 3   route_id           11221 non-null  object        
 4   route_short_name   11221 non-null  object        
 5   service_date       11221 non-null  datetime64[ns]
 6   num_sched          11221 non-null  int64         
 7   num_vp             11221 non-null  int64         
 8   pct_w_vp           11221 non-null  float64       
 9   weekday            11221 non-null  object        
 10  month              11221 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 964.4+ KB


In [9]:
rt_sched.agency_name.value_counts()

SamTrans        6936
Big Blue Bus    1002
Name: agency_name, dtype: int64

In [10]:
rt_sched.calitp_itp_id.value_counts()

290    10219
300     1002
Name: calitp_itp_id, dtype: int64

## Charting

In [11]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

import seaborn as sns


In [12]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,num_sched,num_vp,pct_w_vp,weekday,month
168,290,SamTrans,1,278,278,2022-05-04,28,0,0.00,Wednesday,May


In [13]:
rt_sched>>count(_.month)

,month,n
0,June,5170
1,May,6051


In [14]:
rt_sched>>group_by(_.agency_name, _.month)>>summarize(avg = _.pct_w_vp.mean())

,agency_name,month,avg
0,Big Blue Bus,June,0.86
1,Big Blue Bus,May,0.85
2,SamTrans,June,0.79
3,SamTrans,May,0.88


In [15]:
#aggregating by date
day_pct = utils.agg_by_date(rt_sched, 'num_sched', 'num_vp')

In [16]:
day_pct

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,290,SamTrans,1,2022-05-01,Sunday,May,1270,937,0.74
1,290,SamTrans,1,2022-05-02,Monday,May,2886,2168,0.75
2,290,SamTrans,1,2022-05-03,Tuesday,May,3021,2952,0.98
3,290,SamTrans,1,2022-05-04,Wednesday,May,2901,3,0.00
4,290,SamTrans,1,2022-05-05,Thursday,May,2886,2160,0.75
...,...,...,...,...,...,...,...,...,...
117,300,Big Blue Bus,0,2022-06-26,Sunday,June,772,500,0.65
118,300,Big Blue Bus,0,2022-06-27,Monday,June,1465,1017,0.69
119,300,Big Blue Bus,0,2022-06-28,Tuesday,June,1559,1457,0.93
120,300,Big Blue Bus,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [17]:
#see if we have multiple date entries
day_pct>>count(_.service_date)>>arrange(-_.n)

,service_date,n
0,2022-05-01,2
1,2022-05-02,2
2,2022-05-03,2
3,2022-05-04,2
4,2022-05-05,2
...,...,...
56,2022-06-26,2
57,2022-06-27,2
58,2022-06-28,2
59,2022-06-29,2


## Percent by Day version 1

In [22]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp:Q', title=utils.labeling('pct_w_vp'), axis=alt.Axis(format='%')),
            color=alt.Color(
                'pct_w_vp',
                scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp')), symbolLimit=10)
            )
        ).properties(title='Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data'))
    
chart = styleguide.preset_chart_config(bar)
chart.properties(width=800)

alt.Chart(...)

In [24]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==st))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp:Q', title=utils.labeling('pct_w_vp'), axis=alt.Axis(format='%')),
            color=alt.Color(
                'pct_w_vp',
                scale=alt.Scale(range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp')), symbolLimit=10)
            )
        ).properties(title='SamTrans Percent of Scheduled Trips with RT Vehicle Position Data'))
    
chart = styleguide.preset_chart_config(bar)
chart.properties(width=800)

alt.Chart(...)

## Percent by Day Version 2

In [25]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==bbb))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title=" Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(rt_sched>>filter(_.calitp_itp_id==bbb)).mark_rule(color='red').encode(
    y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%'))
)
## add line
# line = alt.Chart(day_pct).mark_line(
#     color='red',
#     size=2
# ).transform_window(
#     rolling_mean='mean(pct_w_vp)',
#     frame=[-7, 7]
# ).encode(
#     x='service_date:T',
#     y='rolling_mean:Q'
# )

(bar + rule).properties(width=900)

alt.LayerChart(...)

In [26]:
bar = (alt.Chart(day_pct>>filter(_.calitp_itp_id==st))
        .mark_bar(size=8)
        .encode(
            x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
            y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
            color=alt.Color(
                'weekday',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS),
                legend=alt.Legend(title=(utils.labeling('pct_w_vp'))))
        ).properties(title="SamTrans Percent of Scheduled Trips with RT Vehicle Position Data"))

# # mean line
rule = alt.Chart(rt_sched>>filter(_.calitp_itp_id==bbb)).mark_rule(color='red').encode(
    y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%'))
)
## add line
# line = alt.Chart(day_pct).mark_line(
#     color='red',
#     size=2
# ).transform_window(
#     rolling_mean='mean(pct_w_vp)',
#     frame=[-7, 7]
# ).encode(
#     x='service_date:T',
#     y='rolling_mean:Q'
# )

(bar + rule).properties(width=900)

alt.LayerChart(...)

In [27]:
day_pct.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
121,300,Big Blue Bus,0,2022-06-30,Thursday,June,1568,1481,0.94


## Percent by Day Version 3

In [58]:
base = alt.Chart(day_pct).properties(width=550)
line = base.mark_line().encode(
    x=alt.X('service_date', title=utils.labeling('service_date'), sort=("x")),
    y=alt.Y('pct_w_vp', title=utils.labeling('pct_w_vp')),
    color='agency_name'
)

# # mean line
rule = alt.Chart(rt_sched>>filter(_.calitp_itp_id==bbb)).mark_rule(color='red').encode(
    y=alt.Y('mean(pct_w_vp):Q', axis=alt.Axis(format='%'))
)

line + rule

alt.LayerChart(...)

## Total Number of Scheduled Trips vs Vehicle Position Trips

In [28]:
df_long =  (day_pct>>select(_.agency_name,
                            _.service_date,
                         _.total_num_sched,
                         _.total_num_vp
                        ) 
             >> gather('measure',
                       'value',
                       _.total_num_sched,
                       _.total_num_vp
                      )
            )

In [29]:
df_long

,agency_name,service_date,measure,value
0,SamTrans,2022-05-01,total_num_sched,1270
1,SamTrans,2022-05-02,total_num_sched,2886
2,SamTrans,2022-05-03,total_num_sched,3021
3,SamTrans,2022-05-04,total_num_sched,2901
4,SamTrans,2022-05-05,total_num_sched,2886
...,...,...,...,...
239,Big Blue Bus,2022-06-26,total_num_vp,500
240,Big Blue Bus,2022-06-27,total_num_vp,1017
241,Big Blue Bus,2022-06-28,total_num_vp,1457
242,Big Blue Bus,2022-06-29,total_num_vp,1454


In [31]:
bar = (alt.Chart(df_long>>filter(_.agency_name == ('Big Blue Bus'))).mark_bar(opacity=1, size=5)
       .mark_bar(size=9)
       .encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date'), axis=alt.Axis(format='%b, %Y')),
    y=alt.Y('value', stack=None,  title=_dla_utils.labeling('number of trips')),
    color="measure")
      ).properties(title=" Big Blue Bus Scheduled Trips with RT Vehicle Position Data")

chart = styleguide.preset_chart_config(bar)
chart.properties(width=900)

alt.Chart(...)

In [32]:
bar = (alt.Chart(df_long>>filter(_.agency_name == ('SamTrans'))).mark_bar(opacity=1, size=5)
       .mark_bar(size=9)
       .encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date'), axis=alt.Axis(format='%b, %Y')),
    y=alt.Y('value', stack=None,  title=_dla_utils.labeling('number of trips')),
    color="measure")
      ).properties(title=" Big Blue Bus Scheduled Trips with RT Vehicle Position Data")

chart = styleguide.preset_chart_config(bar)
chart.properties(width=900)

alt.Chart(...)

## By Weekday

In [35]:
bbb_week = (day_pct>>filter(_.calitp_itp_id==bbb))>>group_by(_.weekday)>>summarize(avg_pct_w_vp = _.pct_w_vp.mean())

In [36]:
bbb_week

,weekday,avg_pct_w_vp
0,Friday,0.91
1,Monday,0.73
2,Saturday,0.70
3,Sunday,0.69
4,Thursday,0.92
5,Tuesday,0.89
6,Wednesday,0.92


In [37]:
cats_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
bbb_week['weekday'] = pd.Categorical(bbb_week['weekday'], categories=cats_day, ordered=True)
bbb_week=bbb_week.sort_values('weekday')          

In [38]:
bbb_week

,weekday,avg_pct_w_vp
1,Monday,0.73
5,Tuesday,0.89
6,Wednesday,0.92
4,Thursday,0.92
0,Friday,0.91
2,Saturday,0.70
3,Sunday,0.69


In [39]:
bar = alt.Chart(bbb_week).mark_bar().encode(
    x=alt.X('weekday', title=utils.labeling('weekday'), sort=cats_day),
    y=alt.Y('avg_pct_w_vp:Q', title=utils.labeling('avg_pct_w_vp'), axis=alt.Axis(format='%')),
    color=alt.Color(
                'weekday',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS))
    ).properties(title=" Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day")
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [40]:
bbb_week_month = ((day_pct>>filter(_.calitp_itp_id==bbb))
    >>group_by(_.month, _.weekday)
    >>summarize(avg_pct_w_vp = _.pct_w_vp.mean()))

In [41]:
bbb_week_month.sample()

,month,weekday,avg_pct_w_vp
8,May,Monday,0.75


In [42]:
bar = alt.Chart(bbb_week_month).mark_bar().encode(
    x=alt.X('weekday', title=utils.labeling('weekday'), sort=cats_day),
    y=alt.Y('avg_pct_w_vp:Q', title=utils.labeling('avg_pct_w_vp'), axis=alt.Axis(format='%')),
        color=alt.Color(
                'weekday',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS)),
    column="month").properties(title="Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Day")
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [43]:
cats_month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [44]:
bar = alt.Chart(bbb_week_month).mark_bar().encode(
    y=alt.Y('weekday', title=utils.labeling('weekday'), sort=cats_day),
    x=alt.X('avg_pct_w_vp:Q', title=utils.labeling('avg_pct_w_vp'), axis=alt.Axis(format='%')),
        color=alt.Color(
                'weekday',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS)),
    row=alt.Row("month", sort=cats_month)
    ).properties(title=" Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Month")
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

In [45]:
line = alt.Chart(bbb_week_month).mark_line().encode(
    x=alt.X('weekday', title=_dla_utils.labeling('weekday'), sort=cats_day),
    y=alt.Y('avg_pct_w_vp:Q', title=_dla_utils.labeling('avg_pct_w_vp'), axis=alt.Axis(format='%')),
        color=alt.Color(
                'month',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS)),
    strokeDash='month',
).properties(title=" Average Big Blue Bus Percent of Scheduled Trips with RT Vehicle Position Data by Month")
line_chart = styleguide.preset_chart_config(line)
line_chart.properties(width=600)

alt.Chart(...)

In [46]:
day_pct>>filter(_.pct_w_vp>.95)

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
2,290,SamTrans,1,2022-05-03,Tuesday,May,3021,2952,0.98
5,290,SamTrans,1,2022-05-06,Friday,May,3024,2958,0.98
9,290,SamTrans,1,2022-05-10,Tuesday,May,3018,2921,0.97
10,290,SamTrans,1,2022-05-11,Wednesday,May,3040,2932,0.96
11,290,SamTrans,1,2022-05-12,Thursday,May,3022,2960,0.98
12,290,SamTrans,1,2022-05-13,Friday,May,3020,2940,0.97
16,290,SamTrans,1,2022-05-17,Tuesday,May,3007,2942,0.98
17,290,SamTrans,1,2022-05-18,Wednesday,May,3034,2963,0.98
18,290,SamTrans,1,2022-05-19,Thursday,May,3017,2958,0.98
19,290,SamTrans,1,2022-05-20,Friday,May,3039,2978,0.98


In [47]:
day_pct>>filter(_.service_date >= '2022-06-01', _.service_date <= '2022-06-03')>>arrange(_.service_date)

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
31,290,SamTrans,1,2022-06-01,Wednesday,June,3056,2990,0.98
92,300,Big Blue Bus,0,2022-06-01,Wednesday,June,1613,1463,0.91
32,290,SamTrans,1,2022-06-02,Thursday,June,3029,2928,0.97
93,300,Big Blue Bus,0,2022-06-02,Thursday,June,1614,1458,0.90
33,290,SamTrans,1,2022-06-03,Friday,June,3027,2925,0.97
94,300,Big Blue Bus,0,2022-06-03,Friday,June,1609,1387,0.86


## Group By Month

In [48]:
utils.groupby_twocol(day_pct, 'agency_name', 'month', 'pct_w_vp', 'month')

,agency_name,month,avg
1,Big Blue Bus,May,0.82
3,SamTrans,May,0.82
0,Big Blue Bus,June,0.83
2,SamTrans,June,0.65


In [51]:
utils.groupby_onecol((day_pct>>filter(_.calitp_itp_id==290)), 'month', 'pct_w_vp')

,month,avg
1,May,0.82
0,June,0.65


In [52]:
bar = alt.Chart(utils.groupby_twocol(day_pct, 'agency_name', 'month', 'pct_w_vp', 'month')).mark_bar().encode(
    x=alt.X('month', title=utils.labeling('month'), sort=cats_month),
    y=alt.Y('avg:Q', title=utils.labeling('pct_w_vp'), axis=alt.Axis(format='%')),
        color=alt.Color(
                'agency_name',
                scale=alt.Scale(range=altair_utils.CALITP_DIVERGING_COLORS)),
    column = 'agency_name')
chart = styleguide.preset_chart_config(bar)
chart

alt.Chart(...)

## Using Functions (without DBT tables)

In [38]:
#gtfs_daily = utils.load_schedule_data(analysis_date_start, analysis_date_end, itp_id)

In [39]:
#gtfs_daily

In [40]:
#gtfs_daily.service_date.min()

In [41]:
#gtfs_daily.service_date.max()

In [42]:
#rt = utils.load_rt_data(analysis_date_start, analysis_date_end)

In [43]:
#rt

In [44]:
#rt['str_len'] = rt.trip_id.str.len()

In [45]:
#rt.str_len.value_counts()

In [46]:
#len(rt)

In [47]:
#rt.trip_id.nunique()

In [48]:
#len(gtfs_daily)

In [49]:
#gtfs_daily.trip_id.nunique()

In [50]:
#date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [51]:

# from datetime import date, timedelta
# start_date = date(2022, 5, 1)
# end_date = date(2022, 5, 31)

In [52]:
# daterange = pd.date_range(start_date, end_date)

In [53]:
# (daterange)

In [54]:
#above range does not work with function

In [55]:
# dates = set()

# def daterange2(start, end):
#     for n in range(int((end - start).days) + 1):
#         yield start + timedelta(n)

# for single_date in daterange2(start_date, end_date):
#     dates.add(single_date.strftime('%Y-%m-%d'))

In [56]:
# (dates)

In [57]:
# rt['date'] = pd.to_datetime(rt['date'])

In [58]:
# gtfs_daily['service_date'] = pd.to_datetime(gtfs_daily['service_date'])

In [59]:
# single_date2 = '2022-05-08'

# (rt>>filter(_.date == single_date2))


In [60]:
# def get_pct_ran_df2(itp_id, list_of_dates):
#     pcts = []
#     for single_date in list_of_dates:
#         gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
#         rt2 = (rt>>filter(_.date == single_date))
#         sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))
#         #pct_ran = (utils.get_pct_ran(sched_rt_df, single_date))
#         day_pct_ran = {}
#         day_pct_ran['date'] = single_date
#         if ((len(sched_rt_df))!=0):
#             day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
#         elif ((len(sched_rt_df))==0):
#             day_pct_ran['pct_trips_ran'] = ''
#         pct_ran = pd.DataFrame([day_pct_ran])
#         pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
#         pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
#         pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
#         pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
#         pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

#         pcts.append(pct_ran)                                                    
#         #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
#     pcts = pd.concat(pcts)
#     pcts = pcts>>arrange(_.date)
#     return pd.DataFrame(pcts)


In [61]:
# may = utils.get_pct_ran_df(itp_id, dates, gtfs_daily, rt)

In [62]:
# may